<a href="https://colab.research.google.com/github/MikeTango2002/Tennis-Analyzer/blob/main/training/tennis_ball_detector_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)

2.3.0+cu121
0.18.0+cu121


In [2]:
!nvidia-smi

Fri May 31 09:48:28 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Bisogna dapprima installare la libreria di roboflow, sito da cui è stato preso il dataset "tennis-ball-detection",
e la libreria ultralytics, la quale consente di utilizzare yolo

In [3]:
!pip install -q roboflow
!pip install -q ultralytics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 872.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.4/779.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 51.4 MB/s eta 0:00:00


In [4]:
import ultralytics
print(ultralytics.__version__)
print(ultralytics.__file__)


8.2.26
/usr/local/lib/python3.10/dist-packages/ultralytics/__init__.py


# Get Dataset

In [5]:
#Codice generate da roboflow per importare il dataset di tennis-ball-detection
from roboflow import Roboflow
rf = Roboflow(api_key="sR7Pv1xXBovkKUMHtq3j")
project = rf.workspace("viren-dhanwani").project("tennis-ball-detection")
version = project.version(6)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.26, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to tennis-ball-detection-6 in yolov8:: 100%|██████████| 1168/1168 [00:00<00:00, 4594.46it/s]


In [6]:
import shutil

shutil.move("tennis-ball-detection-6/train",
"tennis-ball-detection-6/tennis-ball-detection-6/train",
)
shutil.move("tennis-ball-detection-6/test",
"tennis-ball-detection-6/tennis-ball-detection-6/test",
)
shutil.move("tennis-ball-detection-6/valid",
"tennis-ball-detection-6/tennis-ball-detection-6/valid",
)

'tennis-ball-detection-6/tennis-ball-detection-6/valid'

In [11]:
print(dataset.location)

/content/tennis-ball-detection-6


Poichè yolo non riconosce bene le palline di tennis, allenerò una rete yolo con un dataset specifico in cui sono presenti dei frame in cui è stata messa in evidenza la posizione della pallina.

In [1]:
from ultralytics import YOLO

# Inizializzare il modello YOLOv8x
model = YOLO("../models/yolov8l6u.pt")

# Addestrare il modello
model.train(data=f"{dataset.location}/data.yaml", epochs=20, imgsz=640)


TypeError: YOLO.__init__() got an unexpected keyword argument 'pretrained'

Una volta addestrato il modello, nella cartella content/ è presente una cartella runs/detect/train che contiene i pesi del modello e anche metriche e grafici sull'andamento dell'addestramento.